imports + mount

In [1]:
import os
import cv2
import re
import numpy as np

 # mount and imports

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
# Directories for RGB and Thermal images
rgb_original_dirs = [
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20230825/Prediction/RGB',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20230929/prediction/raw data/RGB/new_name',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20231103/prediction/raw data/RGB',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20231216/prediction/raw data/RGB',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20240830/raw data/RGB',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20240927/raw data/RGB/new_name',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20241102/raw data/RGB/new_name',
    '/content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/RGB/new_name'

]

thermal_original_dirs = [
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20230825/Prediction/tiff',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20230929/prediction/raw data/tiff',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20231103/prediction/raw data/tiff',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20231216/prediction/raw data/tiff',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20240830/raw data/tiff',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20240927/raw data/tiff',
    # '/content/drive/Othercomputers/My PC/Thesis/raw_data/20241102/raw data/tiff',
    '/content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/tiff/new_name'
]

# Mask directories
# rgb_mask_dir = '/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_automate_mask_ganarator/RGB'
rgb_mask_dir = '/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_automate_mask_ganarator/RGB_small_resulotion'
thermal_mask_dir = '/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_automate_mask_ganarator/Thermal'

# Output directories
output_dirs = {
    "2023_rgb": '/content/drive/Othercomputers/My PC/Thesis/Segmented_by_SAM/2023/RGB',
    "2024_rgb": '/content/drive/Othercomputers/My PC/Thesis/Segmented_by_SAM/2024/RGB',
    "2023_thermal": '/content/drive/Othercomputers/My PC/Thesis/Segmented_by_SAM/2023/tiff',
    "2024_thermal": '/content/drive/Othercomputers/My PC/Thesis/Segmented_by_SAM/2024/tiff'
}


In [24]:
import re

def extract_year_from_path(path):
    year_pattern = re.compile(r'\b(202[34]\d{4})\b')  # Extract YYYYMMDD
    match = year_pattern.search(path)
    if match:
        full_date = match.group(0)  # "20241129"
        year = full_date[:4]        # "2024"
        return year, full_date  # ✅ Return both
    return None, None



In [19]:
def process_images(original_dirs, mask_dir, image_type):
    for original_dir in original_dirs:
        print(f"Processing directory: {original_dir}")
        # Extract the year dynamically from the path
        year = extract_year_from_path(original_dir)

        if not year:
            print(f"Unable to extract year from path: {original_dir}")
            continue

        # Construct the output directory key (e.g., '2023_rgb' or '2024_thermal')
        output_dir_key = f"{year}_{'rgb' if image_type == 'RGB' else 'thermal'}"

        # Get the corresponding output directory
        output_dir = output_dirs.get(output_dir_key)

        if output_dir is None:
            print(f"Invalid directory key: {output_dir_key}")
            continue

        # List of original images
        original_images = [f for f in os.listdir(original_dir) if f.endswith(('JPG', 'tiff'))]

        # Process each original image
        for original_image in original_images:
            print(f"Processing image: {original_image} in {original_dir}")
            # Load the original image
            original_image_path = os.path.join(original_dir, original_image)
            original = cv2.imread(original_image_path)

            # Find the corresponding mask file
            if image_type == 'RGB':
                mask_name = original_image.replace('.JPG', '_masked.png')  # RGB images
            else:
                # For thermal images, handle 2023 and 2024 naming differently
                if year == '2023':
                    mask_name = original_image.replace('.tiff', '_masked.tiff')  # Thermal 2023: FLIR0819.tiff -> FLIR0819_masked.tiff
                else:
                    mask_name = original_image.replace('.tiff', '_masked.tiff')  # Thermal 2024: IR_30082024_0373.tiff -> IR_30082024_0373_masked.tiff

            mask_path = os.path.join(mask_dir, mask_name)
            print(f"Looking for mask: {mask_path}")  # <-- Debug print statement

            if not os.path.exists(mask_path):
                print(f"Mask not found for {original_image}")
                continue


            # Check if the processed image already exists
            save_name = os.path.join(output_dir, original_image)
            if os.path.exists(save_name):
                print(f"Processed image already exists for {original_image}, skipping...")
                continue

            # Load the mask (binary image, white = object, black = background)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # Ensure the mask is binary (0 for background, 255 for object)
            mask_binary = np.where(mask > 0, 255, 0).astype(np.uint8)

            # Create a white background image
            white_background = np.full_like(original, 255)

            # Apply the mask to the original image
            masked_image = cv2.bitwise_and(original, original, mask=mask_binary)

            # Combine the masked image with the white background
            final_image = np.where(mask_binary[:, :, None] == 0, white_background, masked_image)

            # Save the final masked image to the corresponding output folder
            cv2.imwrite(save_name, final_image)
            print(f"Processed and saved: {save_name}")


# Process RGB images
process_images(rgb_original_dirs, rgb_mask_dir, image_type='RGB')

# # Process Thermal images
process_images(thermal_original_dirs, thermal_mask_dir, image_type='Thermal')


Processing directory: /content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/RGB/new_name
Invalid directory key: ('2024', '20241129')_rgb
Processing directory: /content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/tiff/new_name
Invalid directory key: ('2024', '20241129')_thermal


get the segmented image from the SAM atomate result mask + save it

In [28]:
import os
import cv2
import numpy as np

def process_images(original_dirs, mask_base_dir, image_type):
    for original_dir in original_dirs:
        print(f"\n📂 Processing directory: {original_dir}")

        # Extract year and full date string
        year, date_str = extract_year_from_path(original_dir)
        if not year or not date_str:
            print(f"❌ ERROR: Unable to extract year or date from path: {original_dir}")
            continue

        print(f"✅ Extracted year: {year}, Extracted date: {date_str}")

        # Determine the correct output directory
        output_dir_key = f"{year}_{'rgb' if image_type == 'RGB' else 'thermal'}"
        output_base_dir = output_dirs.get(output_dir_key)

        if output_base_dir is None:
            print(f"❌ ERROR: Invalid output directory key: {output_dir_key}")
            continue

        # Create a date-specific folder within the output directory
        output_dir = os.path.join(output_base_dir, date_str)
        os.makedirs(output_dir, exist_ok=True)

        # List all original images
        original_images = [f for f in os.listdir(original_dir) if f.endswith(('JPG', 'tiff'))]

        for original_image in original_images:
            print(f"\n🖼️ Processing image: {original_image} in {original_dir}")

            # Construct the save path and skip if already processed
            save_name = os.path.join(output_dir, original_image)
            if os.path.exists(save_name):
                print(f"⚠️ Processed image already exists for {original_image}. Skipping...")
                continue

            # Load the original image
            original_image_path = os.path.join(original_dir, original_image)
            original = cv2.imread(original_image_path)

            if original is None:
                print(f"❌ ERROR: Failed to load original image: {original_image_path}")
                continue

            # Construct mask directory
            if image_type == 'RGB':
                mask_folder = os.path.join(rgb_mask_dir, date_str)
                mask_name = original_image.replace('.JPG', '_masked.png')
            else:
                mask_folder = os.path.join(thermal_mask_dir, date_str)
                mask_name = original_image.replace('.tiff', '_masked.tiff')

            # Debugging: Check paths
            print(f"🔍 Looking for mask...")
            print(f"   📂 Mask folder: {mask_folder}")
            print(f"   🎭 Expected mask name: {mask_name}")

            # Check if the mask folder exists
            if not os.path.exists(mask_folder):
                print(f"❌ ERROR: Mask folder does NOT exist: {mask_folder}")
                continue

            # Check for different file name variations
            mask_path = os.path.join(mask_folder, mask_name)
            mask_variants = [
                mask_path,
                mask_path.replace('.tiff', '.tif'),   # Try ".tif"
                mask_path.replace('.tiff', '.TIFF'), # Try uppercase ".TIFF"
                os.path.join(mask_folder, mask_name.replace('_masked', ''))  # Try without "_masked"
            ]

            # Try each variation
            found_mask = None
            for variant in mask_variants:
                if os.path.exists(variant):
                    found_mask = variant
                    break

            if found_mask is None:
                print(f"❌ ERROR: Mask not found for {original_image}")
                continue
            else:
                print(f"✅ Mask found: {found_mask}")

            # Load the mask
            mask = cv2.imread(found_mask, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                print(f"❌ ERROR: Failed to load mask: {found_mask}")
                continue

            # Ensure the mask is binary and has the same dimensions as the original image
            mask_binary = np.where(mask > 0, 255, 0).astype(np.uint8)

            # Resize the mask if dimensions don't match
            if mask.shape[:2] != original.shape[:2]:
                print(f"⚠️ Resizing mask to match original image dimensions for {original_image}")
                mask_binary = cv2.resize(mask_binary, (original.shape[1], original.shape[0]), interpolation=cv2.INTER_NEAREST)

            # Create a white background
            white_background = np.full_like(original, 255)

            # Apply the mask
            try:
                masked_image = cv2.bitwise_and(original, original, mask=mask_binary)
                final_image = np.where(mask_binary[:, :, None] == 0, white_background, masked_image)

                # Save the final masked image
                cv2.imwrite(save_name, final_image)
                print(f"✅ Processed and saved: {save_name}")
            except Exception as e:
                print(f"❌ ERROR: Error processing image {original_image}: {e}")

# Process RGB images
process_images(rgb_original_dirs, rgb_mask_dir, image_type='RGB')

# Process Thermal images
process_images(thermal_original_dirs, thermal_mask_dir, image_type='Thermal')



📂 Processing directory: /content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/RGB/new_name
✅ Extracted year: 2024, Extracted date: 20241129

🖼️ Processing image: 1_e_4_20241129_RGB.JPG in /content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/RGB/new_name
⚠️ Processed image already exists for 1_e_4_20241129_RGB.JPG. Skipping...

🖼️ Processing image: 1_e_5_20241129_RGB.JPG in /content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/RGB/new_name
⚠️ Processed image already exists for 1_e_5_20241129_RGB.JPG. Skipping...

🖼️ Processing image: 1_e_7_20241129_RGB.JPG in /content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/RGB/new_name
⚠️ Processed image already exists for 1_e_7_20241129_RGB.JPG. Skipping...

🖼️ Processing image: 2_e_1_20241129_RGB.JPG in /content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/RGB/new_name
⚠️ Processed image already exists for 2_e_1_20241129_RGB.JPG. Skipping...

🖼️ Processing image: 2

In [ ]:
def process_images(original_dirs, mask_base_dir, image_type):
    for original_dir in original_dirs:
        print(f"Processing directory: {original_dir}")

        # Extract year and date from the path
        year, date_str = extract_year_from_path(original_dir)
        if not year or not date_str:
            print(f"Unable to extract year or date from path: {original_dir}")
            continue

        # Determine the correct output directory
        output_dir_key = f"{year}_{'rgb' if image_type == 'RGB' else 'thermal'}"
        output_base_dir = output_dirs.get(output_dir_key)
        if output_base_dir is None:
            print(f"Invalid output directory key: {output_dir_key}")
            continue

        # Create a date-specific folder within the output directory
        output_dir = os.path.join(output_base_dir, date_str)
        os.makedirs(output_dir, exist_ok=True)

        # List of original images
        original_images = [f for f in os.listdir(original_dir) if f.endswith(('JPG', 'tiff'))]

        # Process each original image
        for original_image in original_images:
            print(f"Processing image: {original_image} in {original_dir}")

            # Construct the save path and skip if the processed image already exists
            save_name = os.path.join(output_dir, original_image)
            if os.path.exists(save_name):
                print(f"Processed image already exists for {original_image}. Skipping...")
                continue

            # Load the original image
            original_image_path = os.path.join(original_dir, original_image)
            original = cv2.imread(original_image_path)

            # Check if the original image was loaded successfully
            if original is None:
                print(f"Failed to load original image: {original_image_path}")
                continue

            # Find the corresponding mask
            if image_type == 'RGB':
                mask_folder = os.path.join(rgb_mask_dir, date_str)
                mask_name = original_image.replace('.JPG', '_masked.png')
            else:
                mask_folder = os.path.join(thermal_mask_dir, date_str)
                mask_name = original_image.replace('.tiff', '_masked.tiff')

            # Handle cases where the mask file has an extra "_masked" suffix
            if not os.path.exists(os.path.join(mask_folder, mask_name)):
                mask_name = mask_name.replace('_masked', '')

            mask_path = os.path.join(mask_folder, mask_name)

            if not os.path.exists(mask_path):
                print(f"Mask not found for {original_image}")
                continue

            # Load the mask
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            # Check if the mask was loaded correctly
            if mask is None:
                print(f"Failed to load mask: {mask_path}")
                continue

            # Ensure the mask is binary and has the same dimensions as the original image
            mask_binary = np.where(mask > 0, 255, 0).astype(np.uint8)

            # Resize mask if its dimensions don't match the original image
            if mask.shape[:2] != original.shape[:2]:
                print(f"Resizing mask to match the original image dimensions for {original_image}")
                mask_binary = cv2.resize(mask_binary, (original.shape[1], original.shape[0]), interpolation=cv2.INTER_NEAREST)

            # Create a white background
            white_background = np.full_like(original, 255)

            # Apply the mask to the original image
            try:
                masked_image = cv2.bitwise_and(original, original, mask=mask_binary)
                final_image = np.where(mask_binary[:, :, None] == 0, white_background, masked_image)

                # Save the final masked image
                cv2.imwrite(save_name, final_image)
                print(f"Processed and saved: {save_name}")
            except Exception as e:
                print(f"Error processing image {original_image}: {e}")

# Process RGB images
process_images(rgb_original_dirs, rgb_mask_dir, image_type='RGB')

# Process Thermal images
process_images(thermal_original_dirs, thermal_mask_dir, image_type='Thermal')


Streaming output truncated to the last 5000 lines.
Processing image: 4_s_7_20230929.JPG in /content/drive/Othercomputers/My PC/Thesis/raw_data/20230929/prediction/raw data/RGB/new_name
Processed and saved: /content/drive/Othercomputers/My PC/Thesis/Segmented_by_SAM/2023/RGB/20230929/4_s_7_20230929.JPG
Processing image: 5_s_1_20230929.JPG in /content/drive/Othercomputers/My PC/Thesis/raw_data/20230929/prediction/raw data/RGB/new_name
Processed and saved: /content/drive/Othercomputers/My PC/Thesis/Segmented_by_SAM/2023/RGB/20230929/5_s_1_20230929.JPG
Processing image: 5_s_2_20230929.JPG in /content/drive/Othercomputers/My PC/Thesis/raw_data/20230929/prediction/raw data/RGB/new_name
Processed and saved: /content/drive/Othercomputers/My PC/Thesis/Segmented_by_SAM/2023/RGB/20230929/5_s_2_20230929.JPG
Processing image: 5_s_3_20230929.JPG in /content/drive/Othercomputers/My PC/Thesis/raw_data/20230929/prediction/raw data/RGB/new_name
Processed and saved: /content/drive/Othercomputers/My PC/Th

MAT files also make mask - need to change the direction every time

In [29]:
import os
import cv2
import scipy.io as sio
import numpy as np
import glob

def extract_year_from_path(path):
    """Extract the year and date string from the given path."""
    parts = path.split('/')
    for part in parts:
        if len(part) == 8 and part.isdigit():
            year = part[:4]
            date_str = part
            return year, date_str
    return None, None

def find_thermal_variable(mat_data):
    """Dynamically find the thermal data variable in the MAT file."""
    for key in mat_data.keys():
        if not key.startswith('__') and isinstance(mat_data[key], np.ndarray):
            return key
    return None

def process_mat_files(original_dirs, mask_base_dir):
    for original_dir in original_dirs:
        print(f"\nProcessing directory: {original_dir}")

        year, date_str = extract_year_from_path(original_dir)
        if not year or not date_str:
            print(f"Unable to extract year or date from path: {original_dir}")
            continue

        output_base_dir = f"/content/drive/Othercomputers/My PC/Thesis/Segmented_by_SAM/{year}/MAT"
        os.makedirs(output_base_dir, exist_ok=True)
        output_dir = os.path.join(output_base_dir, date_str)
        os.makedirs(output_dir, exist_ok=True)

        original_mat_files = [f for f in os.listdir(original_dir) if f.endswith('.MAT')]

        for original_mat_file in original_mat_files:
            print(f"\nProcessing MAT file: {original_mat_file} in {original_dir}")

            save_name = os.path.join(output_dir, original_mat_file.replace('.MAT', '_masked.MAT'))
            if os.path.exists(save_name):
                print(f"Processed MAT file already exists for {original_mat_file}. Skipping...")
                continue

            original_mat_path = os.path.join(original_dir, original_mat_file)
            try:
                mat_data = sio.loadmat(original_mat_path)
                thermal_var_name = find_thermal_variable(mat_data)

                if not thermal_var_name:
                    print(f"No valid thermal data found in {original_mat_file}")
                    continue

                thermal_image = mat_data[thermal_var_name]
                print(f"Loaded thermal data with type: {thermal_image.dtype}")

                if thermal_image.dtype == np.float64:
                    thermal_image = (thermal_image / np.max(thermal_image) * 65535).astype(np.uint16)
                elif thermal_image.dtype != np.uint16:
                    print(f"Unsupported data type {thermal_image.dtype} for {original_mat_file}")
                    continue

            except Exception as e:
                print(f"Failed to load MAT file: {original_mat_file}. Error: {e}")
                continue

            # Correct the mask pattern
            mask_name = original_mat_file.replace('.MAT', '.tiff')
            mask_folder = os.path.join(mask_base_dir, date_str)
            mask_pattern = os.path.join(mask_folder, mask_name)

            print(f"Looking for mask with pattern: {mask_pattern}")

            # Use glob to find the mask
            mask_files = glob.glob(mask_pattern)

            if not mask_files:
                print(f"Mask not found for {original_mat_file} with pattern {mask_pattern}")
                continue

            mask_path = mask_files[0]
            print(f"Found mask: {mask_path}")

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                print(f"Failed to load mask: {mask_path}")
                continue

            mask_binary = np.where(mask > 0, 1, 0).astype(np.uint8)

            if mask.shape[:2] != thermal_image.shape[:2]:
                print(f"Resizing mask to match the MAT file dimensions for {original_mat_file}")
                mask_binary = cv2.resize(mask_binary, (thermal_image.shape[1], thermal_image.shape[0]), interpolation=cv2.INTER_NEAREST)

            masked_thermal_image = np.where(mask_binary == 1, thermal_image, 0)

            try:
                sio.savemat(save_name, {thermal_var_name: masked_thermal_image})
                print(f"Processed and saved: {save_name}")
            except Exception as e:
                print(f"Error saving MAT file {original_mat_file}: {e}")

# Define the original directories containing MAT files
mat_original_dirs = [
    '/content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/Thermal MAT/new_name'
]

# Define the mask base directory
thermal_mask_dir = '/content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_automate_mask_ganarator/Thermal'

process_mat_files(mat_original_dirs, thermal_mask_dir)



Processing directory: /content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/Thermal MAT/new_name

Processing MAT file: 1_e_4_20241129_Thermal.MAT in /content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/Thermal MAT/new_name
Loaded thermal data with type: float64
Looking for mask with pattern: /content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_automate_mask_ganarator/Thermal/20241129/1_e_4_20241129_Thermal.tiff
Found mask: /content/drive/Othercomputers/My PC/Thesis/roboflowV6_1169/SAM_automate_mask_ganarator/Thermal/20241129/1_e_4_20241129_Thermal.tiff
Processed and saved: /content/drive/Othercomputers/My PC/Thesis/Segmented_by_SAM/2024/MAT/20241129/1_e_4_20241129_Thermal_masked.MAT

Processing MAT file: 1_e_5_20241129_Thermal.MAT in /content/drive/Othercomputers/My PC/Thesis/raw_data/20241129/raw data/Thermal MAT/new_name
Loaded thermal data with type: float64
Looking for mask with pattern: /content/drive/Othercomputers/My PC/Thesis/roboflow